In [ ]:
import matplotlib.pyplot as plt
import datetime
import matplotlib.dates as mdates
from utils import *

In [ ]:
q = 0
K = 100
maturity_c_dict = {'case1': [10, 0.15/2, 20, 'Llyods Banking Group (1)'],
                  'case2': [5, 7.875/100/2, 10, 'Credit Suisse'],
                  'case3': [5, 7/100/4, 20, 'Llyods Banking Group (2)'],
                  'case4': [7, 7.25/100/2, 14, 'Credit Suisse (Gov Intervention)'], # has government intervention
                  'case5': [5, 4.22/100, 5, 'CCB']
                  }



#for process_case in ['case1', 'case2', 'case3', 'case4']:
for process_case in ['case1','case2', 'case3', 'case4', 'case5']:
    
    # [read parameterrs]
    stock_param = pd.read_csv('../param/J_' + process_case + '.csv')
    coco_param = pd.read_csv('../param/CoCo_' + process_case + '.csv')

    
    if process_case =='case4':
        loss, mu, sigma, l2, muV, sigmaV, eta, k1, xi1, k2, xi2, l32, l1, b, a = stock_param.iloc[0].values  
        ignore_gov = False
        pricing_period = ['1/1/2020', '3/17/2023']
    else:
        loss, mu, sigma, l2, muV, sigmaV, eta, l1, b, a = stock_param.iloc[0].values
        k1, xi1, k2, xi2, l32, ignore_gov = [None, None, None, None, None, True]
        pricing_period = None
    loss_coco, p, w, Jbar, wbar = coco_param.iloc[0].values
    
    
    
    # [read data]
    data = pd.read_excel('../data/Charlie1124/'+ process_case + '/data-'+ process_case + '.xlsx')
    data = data.set_index('Date')
    
    if pricing_period == None:
        pricing_period = [datetime.datetime.strftime(data.index[0], format = "%Y-%m-%d"), 
                          datetime.datetime.strftime(data.index[-1], format = "%Y-%m-%d")]

    date_flag = data.index.isin(pd.date_range(start=pricing_period[0], end=pricing_period[1]))
    data = data[date_flag]
     
    data = data.dropna(how = 'any')
    St = data['Stock'].values
    RET= data['return without dividend']/100
    r = data['r'].values/100
    coco_price = data['CoCo'].values
    T, c, M, title= maturity_c_dict[process_case]
    maturity = pd.to_datetime(data.index[0]) + pd.DateOffset(years = T)
    t0 = np.array(range(data.index.shape[0]))/252
    
    
    RET_grids = np.linspace(-0.15, 0.15, 100)
    Eval_Density = Density_stock(l1, a, b, mu, sigma, l2, muV, sigmaV, eta, RET_grids)
    Data_DensityStock = KDE_estimate(RET.values, RET_grids)
    
    
    
    fig= plt.figure(figsize = (18,6))
    ax = fig.add_subplot(121)
    plt.plot(RET_grids, Eval_Density, linestyle='--', label='Fitted')
    plt.plot(RET_grids, Data_DensityStock, label='Kernel')
    plt.legend()
    ax.set_title(title + ' (' + pricing_period[0] + ' -- ' + pricing_period[1] + ')')
    

 
    model_price = equityconvert_coco(r, K, T, t0,
                                     l1, a, b,
                                     c, eta, p, q,
                                     Jbar, M, w, wbar,
                                     k1, xi1, k2, xi2,
                                     l2, l32, muV, sigmaV, sigma,
                                     ignore_gov=ignore_gov, St = St)

    rmse_loss = np.sqrt(np.mean((model_price - coco_price) ** 2))

    ax = fig.add_subplot(122)
    ax.plot(data.index, coco_price, label = 'actual',  linewidth=2.0)
    ax.plot(data.index, model_price,  label = 'estimated', linewidth=2.0)
    ax.legend()
    ax.set_ylabel('CoCo price')
    ax.set_xlabel('Date')
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%Y'))
    ax.set_title(title + ' (' + pricing_period[0] + ' -- ' + pricing_period[1] + ')' + '---rmse = ' + str(rmse_loss.round(4)))
    #fig.savefig('../figure/llyods/llyods_casestudy.png', format='png', dpi=200)
    plt.show()

In [10]:

process_case = 'case4'
stock_param = pd.read_csv('../param/J_' + process_case + '.csv')
coco_param = pd.read_csv('../param/CoCo_' + process_case + '.csv')

loss, mu, sigma, l2, muV, sigmaV, eta, k1, xi1, k2, xi2, l32, l1, b, a = stock_param.iloc[0].values  
ignore_gov = False
pricing_period = ['1/1/2020', '3/17/2023']


data = pd.read_excel('../data/Credit_Suisse_Data_13-23.xlsx', sheet_name='Returns')
data = data.set_index('Names Date')

RET = data['Returns without Dividends']

RET_grids = np.linspace(-0.15, 0.15, 100)
Eval_Density = Density_stock(l1, a, b, mu, sigma, l2, muV, sigmaV, eta, RET_grids)
Data_DensityStock = KDE_estimate(RET.values, RET_grids)


plt.plot(RET_grids, Eval_Density, linestyle='--', label='Fitted')
plt.plot(RET_grids, Data_DensityStock, label='Kernel')
plt.legend()
ax.set_title(title + ' (' + pricing_period[0] + ' -- ' + pricing_period[1] + ')')